In [ ]:
import os
import sys
import shutil
from Format import YOLO
import pandas as pd
import numpy as np
from PIL import Image

In [ ]:
def get_data(csv_path, img_path):
    try:
        df = pd.read_csv(csv_path)
        data = {}
        label_counter = 0
        for idx, row in df.iterrows():
            filename = row[0]
            if row[0] not in data:
                label_counter = 0
                img = Image.open(os.path.join(img_path, filename))
                img_width = str(img.size[0])
                img_height = str(img.size[1])
                img_depth = 3

                size = {
                    "width": img_width,
                    "height": img_height,
                    "depth": img_depth
                }
                data[filename] = {
                    "size": size,
                    "objects": {}
                }
            bndbox = {
                "xmin": row[2],
                "xmax": row[3],
                "ymin": row[4],
                "ymax": row[5]
            }

            cls = row[1]

            obj_info = {
                "name": cls,
                "bndbox": bndbox
            }

            obj = {
                "num_obj": label_counter + 1,
                str(label_counter): obj_info
            }

            data[filename]["objects"] = {**data[filename]["objects"], **obj}

            label_counter = label_counter + 1
        return True, data
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]

        msg = "ERROR : {}, moreInfo : {}\t{}\t{}".format(e, exc_type, fname, exc_tb.tb_lineno)

        return False, msg

In [ ]:
def prepare_val_dataset(src_csv_path, src_img_path, output_csv_path, output_img_path):
    shutil.rmtree(output_img_path)
    os.makedirs(output_img_path)
    if os.path.isfile(output_csv_path):
        os.remove(output_csv_path)
    df = pd.read_csv(src_csv_path)
    img_names = df['ImageID'].unique()
    random_imgs = np.random.choice(img_names, 100, False)
    flipped_df = df.loc[df['ImageID'].isin(random_imgs)].reset_index(drop=True)
    for filename in random_imgs:
        img = Image.open(os.path.join(src_img_path, filename))
        rotated = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        rotated.save(os.path.join(output_img_path, filename))
        flipped_df.loc[flipped_df['ImageID'] == filename, 'Width'] = img.size[0]
    flipped_df['Width'] = flipped_df['Width'].astype(int)
    flipped_df['XMin'] = flipped_df['Width'] - flipped_df['XMin']
    flipped_df['XMax'] = flipped_df['Width'] - flipped_df['XMax']
    flipped_df['XMin'] = flipped_df['XMin'].astype(str)
    flipped_df['XMax'] = flipped_df['XMax'].astype(str)
    flipped_df.to_csv(output_csv_path, index=False)

In [ ]:
def main(config):

    if config["datasets"] == "Dataverse":
        yolo = YOLO(os.path.abspath(config["cls_list"]))

        flag, data = get_data(config["label"], config["img_path"])
        if flag == True:
            flag, data = yolo.generate(data)

            if flag == True:
                flag, data = yolo.save(data, config["output_path"], config["img_path"],
                                        config["img_type"], config["manifest_path"])

                if flag == False:
                    print("Saving Result : {}, msg : {}".format(flag, data))

            else:
                print("YOLO Generating Result : {}, msg : {}".format(flag, data))

        else:
            print("BDD Parsing Result : {}, msg : {}".format(flag, data))

    else:
        print("Unkwon Datasets")

if __name__ == '__main__':

    config1 = {
        "datasets": "Dataverse",
        "img_path": "./datasets/images/train",
        "label": "./datasets/labels.csv",
        "img_type": ".jpg",
        "manifest_path": "./",
        "output_path": "./datasets/labels/train/",
        "cls_list": "./data/dataverse.names",
    }
    config2 = {
        "datasets": "Dataverse",
        "img_path": "./datasets/images/val",
        "label": "./datasets/labels_val.csv",
        "img_type": ".jpg",
        "manifest_path": "./",
        "output_path": "./datasets/labels/val/",
        "cls_list": "./data/dataverse.names",
    }

    main(config1)
    prepare_val_dataset('datasets/labels.csv', 'datasets/images/train', 'datasets/labels_val.csv', 'datasets/images/val')
    main(config2)